# Preprocesamiento del Datatset

En este Jupyter Notebook se preprocesará el texto y después se convertirá a embeddings con un Transformer preentrenado en tweets para clasificación de sentimientos.

In [ ]:
import numpy as np
from joblib import dump
import pandas as pd
pd.set_option('display.max_colwidth', 180)

## Leer los datos

El dataset se eleigió de Kaggle https://www.kaggle.com/code/shtrausslearning/twitter-emotion-classification/notebook

El cual ya se limpió previamente, por lo que solo cambiaremos algunas contracciones para que la conversión a embeddings sea mejor.

In [ ]:
test = pd.read_csv('test.csv')
train = pd.read_csv('training.csv')
validation = pd.read_csv('validation.csv')

print('Test shape:', test.shape)
print('Train shape:', train.shape)
print('Validation shape:', validation.shape)

Test shape: (2000, 2)
Train shape: (16000, 2)
Validation shape: (2000, 2)


In [ ]:
test.head(10)

,text,label
0,im feeling rather rotten so im not very ambitious right now,0
1,im updating my blog because i feel shitty,0
2,i never make her separate from me because i don t ever want her to feel like i m ashamed with her,0
3,i left with my bouquet of red and yellow tulips under my arm feeling slightly more optimistic than when i arrived,1
4,i was feeling a little vain when i did this one,0
5,i cant walk into a shop anywhere where i do not feel uncomfortable,4
6,i felt anger when at the end of a telephone call,3
7,i explain why i clung to a relationship with a boy who was in many ways immature and uncommitted despite the excitement i should have been feeling for getting accepted into the...,1
8,i like to have the same breathless feeling as a reader eager to see what will happen next,1
9,i jest i feel grumpy tired and pre menstrual which i probably am but then again its only been a week and im about as fit as a walrus on vacation for the summer,3


Cambio de contracciones

In [ ]:
import re

# Dictionary of replacements
replacements = {
    "i" : "I",
    "im" : "I am",
    "i m" : "I am",
    "hadn t" : "had not",
    "hasn t" : "has not",
    "haven t" : "have not",
    "don t" : "do not",
    "can t" : "cannot",
    "didn t" : "did not",
    "aren t" : "are not",
    "isn t" : "is not",
    "it s" : "it is",
    "ive" : "I have",
    "it d" : "it would",
    "how d" : "how did",
    "could ve" : "could have",
    "cuz" : "because",
    "gotta" : "got to",
    "kinda" : "kind of",
    "lemme" : "let me",
    "y know" : "you know",
    "you ll" : "you will",
    "you d" : "you had",
    "why s" : "why is",
    "why re" : "why are",
    "won t" : "will not",
    "would ve": "would have",
    "til" : "until",
    "tis" : "it is",
    "somebody s": "somebody is",
    "someone s": "someone is",
    "mine s" : "mine is"
}

# Function to apply replacements
def replace_words(text, replacements):
    pattern = re.compile(r'\b(' + '|'.join(re.escape(key) for key in replacements.keys()) + r')\b', re.IGNORECASE)
    return pattern.sub(lambda x: replacements[x.group().lower()], text)

# Remove the contractions in each DataFrame
test['text'] = test['text'].apply(replace_words, replacements=replacements)
train['text'] = train['text'].apply(replace_words, replacements=replacements)
validation['text'] = validation['text'].apply(replace_words, replacements=replacements)

In [ ]:
test.head()

,text,label
0,I am feeling rather rotten so I am not very ambitious right now,0
1,I am updating my blog because I feel shitty,0
2,I never make her separate from me because I do not ever want her to feel like I m ashamed with her,0
3,I left with my bouquet of red and yellow tulips under my arm feeling slightly more optimistic than when I arrived,1
4,I was feeling a little vain when I did this one,0


### Duplicated

In [ ]:
a = train[train.duplicated('text')].count()['text']
b = validation[validation.duplicated('text')].count()['text']
c = test[test.duplicated('text')].count()['text']

print(f'Train: {a}')
print(f'Validation: {b}')
print(f'Test: {c}')

Train: 33
Validation: 2
Test: 0


Eliminar duplicados ya que no representa una clasificación múltiple de sentimientos significante.

In [ ]:
train.shape

(15967, 2)

In [ ]:
train = train.drop_duplicates(subset='text')
validation = validation.drop_duplicates(subset='text')

train.to_csv('training.csv', index=False)
validation.to_csv('validation.csv', index=False)

## Embeddings

Ahora convertiremos el texto a embeddings con el siguiente Transformer https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model = AutoModel.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

## Notas personales embeddings

- Dividir el texto en tokens. `tokenizer` Convierte cada token en un número (su ID en el vocabulario del modelo). Añadiendo tokens especiales: ([CLS] al inicio y [SEP] al final) Cuenta con los siguientes parametros:
    - `padding=True`: Rellena con tokens especiales [PAD] hasta que todos los textos del batch tengan la misma longitud
    - `truncation=True`: Corta los textos que excedan max_length
    - `max_length=512`: Longitud máxima permitida
    - `return_tensors="pt"`: Devuelve tensores de PyTorch

- El resultado encoded es un diccionario que contiene:
    - `input_ids`: IDs numéricos de cada token
    - `attention_mask`: 1s para tokens reales, 0s para padding

- El modelo acepta este diccionario y procesa los tokens a través de varias capas de transformers. Cada token interactúa con todos los demás tokens mediante self-attention. Generando representaciones contextuales para cada token


- `outputs.last_hidden_state[:, 0, :]` obtiene el embedding del token [CLS}, ya que tiene la forma de `[batch_size, sequence_length, hidden_size]`
    - primer índice: selecciona todos los elementos del batch
    - segundo índice: selecciona solo el primer token ([CLS])
    - tercer índice: selecciona todas las dimensiones del embedding

- El token [CLS] es importante porque el modelo está entrenado para que este token capture la información semántica de toda la secuencia


Prueba conversión embeddings

In [ ]:
text = "I love programming!"

# Tokenize the text
res = tokenizer(text)
print(res)
# Tokens
tokens = tokenizer.tokenize(text)
print(tokens)
# Ids
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)
# Decode
decoded = tokenizer.decode(ids)
print(decoded)

{'input_ids': [0, 100, 657, 8326, 328, 2], 'attention_mask': [1, 1, 1, 1, 1, 1]}
['I', 'Ġlove', 'Ġprogramming', '!']
[100, 657, 8326, 328]
I love programming!


In [ ]:
def get_embeddings(df, tokenizer, model, batch_size, output_file=None):
    model.eval()  # Poner el modelo en modo evaluación
    embeddings = []
    texts = df['text'].tolist()

    # Procesar los textos en batches
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]

        # Tokenizar el batch completo
        encoded = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

        # Obtener embeddings
        with torch.no_grad():
            outputs = model(**encoded)

        # Obtener embeddings del token [CLS]
        batch_embeddings = outputs.last_hidden_state[:, 0, :].numpy()
        embeddings.append(batch_embeddings)

    # Concatenar todos los embeddings
    embeddings = np.vstack(embeddings)

    # Guardar embeddings si se especifica un archivo de salida
    if output_file:
        dump(embeddings, output_file)
        print(f"Embeddings guardados en: {output_file}")

    return embeddings

In [ ]:
test_embeddings = get_embeddings(test, tokenizer, model, batch_size=32,output_file='test_embeddings.pkl')
train_embeddings = get_embeddings(train, tokenizer, model, batch_size=32,output_file='train_embeddings.pkl')
validation_embeddings = get_embeddings(validation, tokenizer, model, batch_size=32,output_file='validation_embeddings.pkl')

Embeddings guardados en: test_embeddings.pkl
Embeddings guardados en: train_embeddings.pkl
Embeddings guardados en: validation_embeddings.pkl


In [ ]:
# Imprimir forma del resultado
print(f"Test embedding shape: {test_embeddings.shape}")
print(f"Train embedding shape: {train_embeddings.shape}")
print(f"Validation embedding shape: {validation_embeddings.shape}")

Test embedding shape: (2000, 768)
Train embedding shape: (15967, 768)
Validation embedding shape: (1998, 768)


In [ ]:
print(f"Test label shape: {test['label'].shape}")
print(f"Train label shape: {train['label'].shape}")
print(f"Validation label shape: {validation['label'].shape}")

Test label shape: (2000,)
Train label shape: (15967,)
Validation label shape: (1998,)
